# Model application 
 In this notebook we will apply linear regression ,support vector regression and randomforestregressor .We will also use hyperparamenter tuning using gridsearchcv.

### IMPORTING LIBRARY TEMPLATE

In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from mlxtend.plotting import plot_linear_regression
 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression


In [86]:
#these csv are the processed data from feature engineering notebook
df=pd.read_csv("X_train.csv")
dfy=pd.read_csv("y_train.csv")
df

,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,12.247694,0.235294,0.75,0.418208,0.366344,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.090909,0.50,1.0,0.8
1,12.109011,0.000000,0.75,0.495064,0.391317,1.0,1.0,1.0,1.0,0.5,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.363636,0.25,1.0,0.8
2,12.317167,0.235294,0.75,0.434909,0.422359,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.727273,0.50,1.0,0.8
3,11.849398,0.294118,0.75,0.388581,0.390295,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.090909,0.00,1.0,0.0
4,12.429216,0.235294,0.75,0.513123,0.468761,1.0,1.0,0.0,1.0,0.5,...,0.0,0.0,1.0,1.0,1.0,0.00000,1.000000,0.50,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,12.072541,0.235294,0.75,0.400718,0.353592,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.636364,0.25,1.0,0.8
1456,12.254863,0.000000,0.75,0.517503,0.453273,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.5,1.0,0.00000,0.090909,1.00,1.0,0.8
1457,12.493130,0.294118,0.75,0.423859,0.379597,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.5,0.16129,0.363636,1.00,1.0,0.8
1458,11.864462,0.000000,0.75,0.434909,0.393688,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.00000,0.272727,1.00,1.0,0.8


In [87]:
Xtrain,Xtest,ytrain,ytest=train_test_split(df,dfy,test_size=0.25,random_state=5)##Splitting data 

In [88]:
##We didnt fill the nan values of electical in feature engineering 
Xtrain["Electrical"].fillna(Xtrain["Electrical"].median(),inplace=True)
Xtest["Electrical"].fillna(Xtest["Electrical"].median(),inplace=True)
Xtest.shape

(365, 79)

# Linear reagression 

In [89]:
##applying linear regression to the data 
reg=LinearRegression()
reg.fit(Xtrain,ytrain)
y=reg.predict(Xtest)
y.shape

(365, 1)

In [90]:
## testing the accuracy of our model ,we are using r2 evaluation 
r2_score(ytest, y)

0.9204297179780264

# Support vector Regression 

In [91]:
## applying svr for our model and hyperparameter tuning with grid search.
clf=SVR()
parameters = [{'C': [1, 10, 100, 1000,10000,100000], 'kernel': ['linear','poly','rbf']}]
grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 3,
                           n_jobs = -1)
grid_search = grid_search.fit(Xtrain, ytrain)


In [92]:
## out all the possible combination we need the best parameter and its score
grid_search.best_params_

{'C': 100000, 'kernel': 'poly'}

In [93]:
 grid_search.best_score_## its better than linear regression since it is robust to outliers 

0.9425912982463531

# Random Forest Regression

In [94]:
## applying RandomForestRegressor for our model and hyperparameter tuning with grid search.
clf1=RandomForestRegressor()
parameters=[{'n_estimators':[50,20,10,4,3,5,25],'max_features':['auto','sqrt'],'max_depth': [10, 20, 30, 40, 50, None]}]
grid= GridSearchCV(estimator = clf1,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 3,
                           n_jobs = -1)
grid= grid.fit(Xtrain, ytrain)

In [95]:
## out all the possible combination we need the best parameter and its score
grid.best_params_

{'max_depth': None, 'max_features': 'auto', 'n_estimators': 25}

In [96]:
 grid.best_score_## The diffence is huge compared to svr 

0.9895131619947003

So the best regressor among the three is randomforestregressor